In [1]:
!pip install requests

In [2]:
!pip install pandas

In [3]:
import requests
import json
import pandas as pd
retList = []


In [4]:
#호출용 파라미터 지정
def getReviewParam(place, theme, pageNum = 1):
    
    
    return {
    "operationName": "getVisitorReviews",
    "variables": {
      "input": {
        "businessId": place['id'],
        "businessType": "restaurant",
        "item": "0",
        "page": pageNum,
        "size": 40,
        "includeContent": True,
        "getUserStats": True,
        "includeReceiptPhotos": True,
        "cidList": place['categoryPath'][0],
        "theme": theme
      },
      "id": place['id']
    },
  "query": "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        class\n        videoId\n        videoOriginSource\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        isReported\n        isSuspended\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      showBookingItemOptions\n      bookingItemName\n      bookingItemOptions\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      isFollowing\n      followerCount\n      followRequested\n      loginIdno\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
  }



#카테고리 현황 조회용 파라미터
def getStatParam(place):
    return {
  "operationName": "getVisitorReviewStats",
  "variables": {
    "businessType": "restaurant",
    "id": place['id'],
    "itemId": "0"
  },
  "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
}



# {
#   "operationName": "getVisitorReviewStats",
#   "variables": {
#     "businessType": "restaurant",
#     "id": "1813392509",
#     "itemId": "0"
#   },
#   "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
# }




def placeProc(place):
    #장소별 ret 초기화
    placeDict = {}

    placeDict['id'] =  place['id']
    placeDict['name'] =  place['name']
    placeDict['contents'] =  []
    
    statParam = getStatParam(place)

    # 리뷰/테마를 읽어오기 위한 api 정의
    review_url = "https://pcmap-api.place.naver.com/graphql"
    review_headers = {
        "referer": "https://pcmap.place.naver.com/",
        "content-type": "application/json",
        "accept": "*/*",
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
    }

    # 테마 목록 가져오기 
    stat_response = requests.post(review_url, headers=review_headers, data=json.dumps(statParam))
    print('stat request', place['name'], stat_response)
    if stat_response.status_code != 200:        
        return placeDict;
    stat_json = json.loads(stat_response.text)

    # 테막 목록 확인
    themes = stat_json['data']['visitorReviewStats']['analysis']['themes']
    # 테마별 리뷰를 불러온다.
    placeDict['contents'] = []
    for theme in themes:
        # total은 제외한다
        if theme['code'] in ['taste','service','price']:
            nextPage = True
            pageNum = 1
            print (pageNum, nextPage)
            while nextPage:
              print (pageNum, nextPage)
              review_param = getReviewParam(place,theme['code'], pageNum)
              review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
              if review_response.status_code != 200: 
                  continue;
              review_json = json.loads(review_response.text)
              reviewBody = review_json['data']['visitorReviews']['items']
              print(theme['code'], len(reviewBody))
              placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))
              if len(reviewBody)==40:
                  pageNum = pageNum + 1
              else:
                  nextPage = False
                  break
            # #반환된 review 건수가 40건인 경우 page 2 한번 더 호출한다.
            # if len(reviewBody)==40:
            #     review_param = getReviewParam(place, theme['code'], 2)
            #     review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
            #     if review_response.status_code != 200: 
            #         continue;
            #     review_json = json.loads(review_response.text)
            #     reviewBody = review_json['data']['visitorReviews']['items']
            #     print(theme['code'], len(reviewBody))
            #     placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))

    return placeDict
    
    

In [5]:
url = 'https://map.naver.com/v5/api/search'
placeName="sootawon"

headers = {
    'authority': 'map.naver.com',
    'content-type': 'application/json',
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
}

### 음식점 목록을 가져온다 (현재 위치 위경도값:searchCoord, query : 검색어)
### 호출건수 : displayCount : 100
params = {
    'lang': 'ko',
    'caller': 'pcweb',
    'type': 'place',
    #'searchCoord': '37.272429000000116,127.13596284980378', #용인
    'query': '수타원 망포',
    'page': 1,
    'displayCount': 1,
    'isPlaceRecommendationReplace': 'false',
}


# clientX: 127.1275729
# clientY: 37.272429
# x: 127.10589175393739
# y: 37.36958999999972
# bounds: 127.09966902902255;37.363723468708145;127.12069754769959;37.37528555368495


In [6]:

response = requests.get(url, headers=headers, params=params)

result = json.loads(response.text)

#호출결과에서 장소에 대한 목록만 읽어온다
placeList = result['result']['place']['list']

In [7]:
len(placeList)

1

In [8]:
print(placeList)

[{'index': '0', 'rank': '1', 'id': '34015652', 'name': '수타원', 'tel': '031-205-6500', 'isCallLink': False, 'virtualTel': '', 'virtualTelDisplay': '', 'ppc': '1', 'category': None, 'categoryPath': [['220036', '220039', '220091', '220861']], 'rcode': '02117105', 'businessStatus': {'requestTime': '202306101556', 'status': {'code': 8, 'text': '24시간 영업', 'emphasis': False, 'description': '24시간 영업', 'detailInfo': ''}, 'businessHours': '202306100000~202306102400', 'breakTime': '', 'lastOrder': ''}, 'naviInfoText': None, 'address': '경기도 수원시 영통구 영통동 982-2', 'roadAddress': '경기도 수원시 영통구 봉영로1482번길 6', 'abbrAddress': '영통동 982-2', 'shortAddress': ['경기 수원시 영통구', '봉영로1482번길 6'], 'display': '<b>수타원</b>', 'telDisplay': '031-205-6500', 'context': ['짬뽕', '짜장면', '찹쌀탕수육', '사진찍기', '간짜장'], 'reviewCount': 695, 'placeReviewCount': 1516, 'ktCallMd': 'c39816587cc2ff0a75af6058950202d0', 'coupon': '0', 'thumUrl': 'https://ldb-phinf.pstatic.net/20211222_6/1640142296897nlwSh_JPEG/KakaoTalk_20211222_120015302_02.jpg', 

In [9]:
# 길이 확인 및 첫 음식점 확인
print(len(placeList))
retList = []
placeList[0]
print(placeList[0]['id'])
cnt = 0
for place in placeList:
    cnt+=1
    # if cnt<50: continue
    retList.append(placeProc(place))
    
    
# retList.append(placeProc(placeList[1]))
# retList.append(placeProc(placeList[2]))

1
34015652
stat request 수타원 <Response [200]>
1 True
1 True
taste 40
2 True
taste 40
3 True
taste 40
4 True
taste 40
5 True
taste 40
6 True
taste 40
7 True
taste 40
8 True
taste 40
9 True
taste 40
10 True
taste 25
1 True
1 True
service 40
2 True
service 0
1 True
1 True
price 10


In [10]:
cnt = 0
last_json = []
for place in retList:
    if cnt>1: continue;
    if len(place['contents'])>0:
        # 'div' 키의 값이 'taste', 'service', 'price' 인 데이터만 저장할 리스트
        last_json.extend(list(filter(lambda x: x.get('div') in ['taste', 'service', 'price'], place['contents'])))

    

In [11]:
last_json

[{'div': 'taste',
  'cont': '짜장면은 맛은 평범한데 양이 많고,\n등심탕수육은 고기에서 냄새가 안나고 맛있어요.\n짜장면은 소스가 부족한데 요청시 더 주시니 참고하세요.\n그리고 갈비짬뽕은 갈비 대 큰게 두개 들어 있어서 가성비가 좋은 거 같아요!'},
 {'div': 'taste',
  'cont': '군만두 한입 베어 물면 육즙이 진하게 느껴지네요. 간도 적당해서 굳이 간장 찍지않아도되네요. 낙지짬뽕의 낙지는 넘질기네요.\n건물에 딸린 주차장은 넘 좁아서 좀 떨어진 곳에 마련된 주차장을 이용해야합니다.'},
 {'div': 'taste',
  'cont': '기본 짬뽕인데 양이 엄청많아서 갈때마다 배부르게 먹을수 있어요!!낙지짬뽕은 낙지가 푸짐하고 쫄깃해서 정말 맛있더라구요 다음에 또먹을거에요\n\n직원분들도 친절하시고 맛있어서 기분좋게 식사했습니다👍👍'},
 {'div': 'taste',
  'cont': '낙지짬뽕,탕수육 강추\n국물개운하니 맛나요 양도많고!!\n재료아낌없이 넣은 느낌이라 만족스럽습니다'},
 {'div': 'taste',
  'cont': '갈비 짬뽕이 시그니처라고 해서 주문해 봤어요 ㅎㅎ처음 먹어보는 갈비짬뽕인데 국물도 진해서 그런지 진짜 맛있더라구요👍🏻 갈비도 큼직하고 양도 너무 많아서 남겼어요 ㅜㅡ 탕수육도 쫀떡하고 튀김옷이 두껍지 않아서 저한테는 맛있게 먹었어요 ~~~~ㅎㅎ  사장님도 엄청 친절하세요 재방문 무조건 추천이에여! '},
 {'div': 'taste',
  'cont': '자주 방문하는 맛집입니다 \n\n24시간이라 언제든지 찾는 중국집으로\n가격,맛, 양 매우 만족입니다'},
 {'div': 'taste', 'cont': '양도 많고 푸짐하고 맛있어요 ~~'},
 {'div': 'taste',
  'cont': '소갈비 짬뽕 양 진짜 많아요. 삼선짜장은 좀 짭니다.\n양 많아서 탕수육은 안시키길 잘했네요~'},
 {'div': 'taste',
  'cont': '여러번 치댄 수제면이라 해물짬

In [12]:
df = pd.DataFrame(last_json)

In [13]:
df

,div,cont
0,taste,"짜장면은 맛은 평범한데 양이 많고,\n등심탕수육은 고기에서 냄새가 안나고 맛있어요...."
1,taste,군만두 한입 베어 물면 육즙이 진하게 느껴지네요. 간도 적당해서 굳이 간장 찍지않아...
2,taste,기본 짬뽕인데 양이 엄청많아서 갈때마다 배부르게 먹을수 있어요!!낙지짬뽕은 낙지가 ...
3,taste,"낙지짬뽕,탕수육 강추\n국물개운하니 맛나요 양도많고!!\n재료아낌없이 넣은 느낌이라..."
4,taste,갈비 짬뽕이 시그니처라고 해서 주문해 봤어요 ㅎㅎ처음 먹어보는 갈비짬뽕인데 국물도 ...
...,...,...
430,price,너무 짜고 가격대가 맛에 비해 많이 비싸네요.\n7천원만해도 충분할 맛입니다.\n너...
431,price,가격이 비싸서 걱정했는데 양이 푸짐하고 맛있어요~
432,price,가격대비 양도 많고 맛나용
433,price,지인짜 맛있어요 얼큰하고 해물도 많고 먼도 수타로 뽑아서 8천원 이란 금액이 아깝지...


In [14]:
df.to_csv('./data/naver_data/{}.csv'.format(placeName))